In [1]:
import handcalcs.render
import forallpeople
forallpeople.environment('structural', top_level=True)
import plotly.graph_objects as go
import math

# Column transitions calculation

### Column loads

In [2]:
%%render
G = 5040 * kN # Dead load
Q = 950 * kN # Live load
N_ult = (1.2 * G + 1.5 * Q) # Ultimate load

<IPython.core.display.Latex object>

### Column above properties

In [3]:
%%render params
DX_above = 350 * mm # Width
DY_above = 1000 * mm # Depth
fc_above = 50 * MPa # Strength

<IPython.core.display.Latex object>

### Column below properties

In [4]:
%%render params
DX_below = 1000 * mm # Width
DY_below = 350 * mm # Depth
fc_below = 50 * MPa # Strength

<IPython.core.display.Latex object>

### Slab/beam properties

In [5]:
%%render params
h = 350 * mm # Overall depth of joint
fc_joint = 40 * MPa # Joint concrete strength

<IPython.core.display.Latex object>

### Plot

In [6]:
col_above_x = [-DX_above.value*1000/2, DX_above.value*1000/2, DX_above.value*1000/2, -DX_above.value*1000/2]
col_above_y = [DY_above.value*1000/2, DY_above.value*1000/2, -DY_above.value*1000/2, -DY_above.value*1000/2]


col_below_x = [-DX_below.value*1000/2, DX_below.value*1000/2, DX_below.value*1000/2, -DX_below.value*1000/2]
col_below_y = [DY_below.value*1000/2, DY_below.value*1000/2, -DY_below.value*1000/2, -DY_below.value*1000/2]

fig = go.Figure()

fig.add_trace(go.Scatter(x=col_above_x, y=col_above_y, fill='toself', name="Column above"))
fig.add_trace(go.Scatter(x=col_below_x, y=col_below_y, fill='toself', name="Column below"))
fig.update_xaxes(showgrid=False, zerolinewidth=2, zerolinecolor='Grey')
fig.update_yaxes(scaleanchor="x", scaleratio=1, showgrid=False, zerolinewidth=2, zerolinecolor='Grey')
fig.show()

### Effective concrete strength

In [7]:
Dc_min = min(DX_above, DY_above, DX_below, DY_below)

def calculate_fce(restraint, ratio, fc_joint, fc_above, fc_below):
    if restraint == "INTERIOR COLUMN":
        f_ce = min( (1.33 - 0.33) / ratio * fc_joint + (0.25 / ratio) * fc_below, fc_above)

    elif restraint == "RESTRAINED ON TWO OPPOSING SIDES":
        f_ce = min( (1.1 - 0.3) / ratio * fc_joint + (0.2 / ratio) * fc_below, fc_above)

    elif restraint == "RESTRAINED ON TWO ADJACENT SIDES":
        f_ce = min( 1.33 * fc_joint, fc_above, fc_below)

In [14]:
%%render
Restraint = "INTERIOR COLUMN"
Dc_min = min(DX_above, DY_above, DX_below, DY_below) # Smallest column cross section
# h/Dc_min limited to minimum 0.33 (Clause 10.8 i and ii)

if Restraint == "INTERIOR COLUMN":
    f_ce = min( (1.33 - 0.33) / (h / Dc_min) * fc_joint + (0.25 / (h / Dc_min)) * fc_below, fc_above)

elif Restraint == "RESTRAINED ON TWO OPPOSING SIDES":
    f_ce = min( (1.1 - 0.3) / (h / Dc_min) * fc_joint + (0.2 / h_Dc) * fc_below, fc_above)

elif Restraint == "RESTRAINED ON TWO ADJACENT SIDES":
    f_ce = min( 1.33 * fc_joint, fc_above)

f_ce # Effective compressive strength of joint

<IPython.core.display.Latex object>

### Bearing check

In [15]:
%%render

phi_bearing = 0.6
A_1 = min(DX_above, DX_below) * min(DY_above, DX_below) # Bearing area
A_2 = DX_below * DY_below # Area of column below

f_bs1 = phi_bearing * 0.9 * f_ce * math.sqrt(A_2 / A_1)
f_bs2 = phi_bearing * 1.8 * f_ce

f_bs = min(f_bs1, f_bs2) # Design bearing stress

<IPython.core.display.Latex object>

### Maximum transition capacity

In [16]:
%%render
phi_N_s =  f_bs * min(A_1, A_2) / 1000 # Maximum transition capacity

<IPython.core.display.Latex object>

In [17]:
if phi_N_s >= N_ult:
    result = "Sufficient bearing capacity"
else:
    result = "Insufficient bearing capacity"

In [18]:
%%render
N_ult
phi_N_s
result

<IPython.core.display.Latex object>

### Additional longitudinal reinforcement through joint

In [19]:
%%render
alpha_1 = 0.85
phi_N_u = phi_bearing * (A_2 * alpha_1 * f_ce) # Concrete squash load capacity


<IPython.core.display.Latex object>

In [20]:
%%render
f_sy = 500 * MPa
phi_N_squash = max(0, N_ult - phi_N_u)
A_s = phi_N_squash * 1000 / (f_sy - alpha_1 * f_ce) # Longitudinal reinforcement

<IPython.core.display.Latex object>